In [1]:
import numpy as np
import pandas as pd

In [2]:
data_dir = '../data'

## Статистика треков

In [3]:
tracks = pd.read_csv('{}/tracks.csv'.format(data_dir), index_col=0)

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
print ('\t\t###Tracks statstics###')
print (' Number of most unpopular track:',tracks[tracks.popular == 1].shape[0], '\n', 
       'Max popularity:', tracks.popular.max(), '\n',
       'Mean popularity:', tracks.popular.mean(), '\n',
       'Median popularity:',tracks.popular.median(), '\n',
       'Number of tracks, where popular>=10:',tracks[tracks.popular >= 10].shape[0], '\n'
       ' Number transactions:', tracks.popular.sum())
tracks.head()

		###Tracks statstics###
 Number of most unpopular track: 638612 
 Max popularity: 13977 
 Mean popularity: 15.997936875138299 
 Median popularity: 1.0 
 Number of tracks, where popular>=10: 166426 
 Number transactions: 19881836


,trackid,popular,artistid,albumid
0,774269,2,3810,76935
1,432553,2,7239,13835
2,1236511,1,133426,446132
3,969465,1,6838,53879
4,302436,6,16816,35620


In [5]:
def get_threholds(tracks, parts=[1, 0]):
    threholds = []
    n = len(parts) - 1
    for i in range(n):
        t1 = round(tracks.popular.quantile(q=parts[i]))
        t2 = round(tracks.popular.quantile(q=parts[i+1]))
        if i == n - 1:
            t2 = 0
        threholds.append((t1, t2))
    return threholds
thrs = get_threholds(tracks, [1, 0.999, 0.98, 0.88, 0])
thrs

[(13977, 2039), (2039, 109), (109, 11), (11, 0)]

In [6]:
tracks.popular.unique().shape

(2839,)

## Статистика исполнителей

In [7]:
artists = pd.read_csv('{}/artists.csv'.format(data_dir), index_col=0)

In [8]:
artists.shape

(178871, 2)

In [8]:
print('\t\t###Artist statistics###')
print (' Number of most unpopular artists:',artists[artists.popular == 1].shape[0], '\n',
       'Max popularity:', artists.popular.max(), '\n'
       ' Mean popularity:', artists.popular.mean(), '\n',
       'Median popularity:',artists.popular.median(), '\n',
       'Number of artists, that popularity>=10:',artists[artists.popular >= 10].shape[0], '\n',
       'Number transactions:', artists.popular.sum())
artists.head()

		###Artist statistics###
 Number of most unpopular artists: 65530 
 Max popularity: 251773 
 Mean popularity: 111.15181331797775 
 Median popularity: 3.0 
 Number of artists, that popularity>=10: 47295 
 Number transactions: 19881836


,artistid,popular
0,58476,1
1,133660,1
2,42343,3
3,103158,1
4,1656,3577


## Статистика альбомов

In [9]:
albums = pd.read_csv('{}/albums.csv'.format(data_dir), index_col=0)
print ('\t\t###Albums statistics###')
print (' Number of the most unpopular albums:',albums[albums.popular == 1].shape[0], '\n',
       'Max popularity:', albums.popular.max(), '\n',
       'Mean popularity:', albums.popular.mean(), '\n',
       'Median popularity:',albums.popular.median(), '\n',
       'Number of albums, that popularity>=10:',albums[albums.popular >= 10].shape[0], '\n',
       'Number transactions:', albums.popular.sum())
albums.head()

		###Albums statistics###
 Number of the most unpopular albums: 166217 
 Max popularity: 46526 
 Mean popularity: 44.37731519715726 
 Median popularity: 2.0 
 Number of albums, that popularity>=10: 106367 
 Number transactions: 19881836


,albumid,popular
0,86118,1
1,362627,3
2,104733,67
3,266531,4
4,60761,46


## Загружаем транзакции

In [10]:
transactions = pd.read_csv('{}/transactions.csv'.format(data_dir), index_col=0)

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
transactions.head()

,pid,trackid
0,822000,0
1,822000,1
2,822000,2
3,822000,3
4,822000,4


In [12]:
print ('Number transactions without duplicates:', transactions.shape[0])

Number transactions without duplicates: 19617679


## Загружаем плейлисты

In [32]:
playlists = pd.read_csv('{}/playlists.csv'.format(data_dir), index_col=0)
playlists.head()

,pid,name,num_followers
0,822000,rap,1
1,822001,wedding mix,1
2,822002,Party,1
3,822003,wElP,1
4,822004,Worship,1


## Посмотрим на статистику названий плейлистов

In [14]:
print ("Number of different names:", playlists.name.unique().shape[0])
grouped = playlists[['pid', 'name']].groupby('name')
grouped.count().sort_values(by='pid', ascending=False).head()

Number of different names: 52531


,pid
name,
Country,1707
Chill,1526
Rap,1477
Christmas,1438
Workout,1437


Приведем к все названия к нижнему регистру

In [15]:
playlists['name'] = [str(playlists['name'].values[i]).lower() for i in range(playlists.shape[0])]
print ("Number of different names:", playlists.name.unique().shape[0])
grouped = playlists[['pid', 'name']].groupby('name')
grouped.count().sort_values(by='pid', ascending=False).head()

Number of different names: 29569


,pid
name,
chill,2803
country,2738
workout,2424
rap,2412
oldies,2249


Оставим в названиях только буквы и цифры

In [16]:
import re
names = playlists['name'].values
for i in range(names.shape[0]):
    names[i] = re.compile('[^a-zA-Z0-9 ]').sub('', names[i])

In [17]:
playlists['name'] = names
print ("Number of different names:", playlists.name.unique().shape[0])
grouped = playlists[['pid', 'name']].groupby('name')
grouped.count().sort_values(by='pid', ascending=False).head(6)

Number of different names: 21663


,pid
name,
,4605
chill,2895
country,2859
workout,2482
rap,2475
oldies,2282


In [18]:
max_len = 0
max_name = names[0]
for name in names:
    if len(name) > max_len:
        max_len = len(name)
        max_name = name
print ('Max len of name:', max_len, '\nname:', max_name)

Max len of name: 72 
name: various artists  fifty shades of grey original motion picture soundtrack


Посмотрим сколько различных слов в названиях и их статистику

In [19]:
words_in_names = []
for i in range(names.shape[0]):
    words_in_names += names[i].split(' ')
words_in_names = np.array(words_in_names)
words = pd.DataFrame({'word': words_in_names, 'id':np.arange(words_in_names.shape[0])})
words.head()

,id,word
0,0,rap
1,1,wedding
2,2,mix
3,3,party
4,4,welp


In [20]:
print ("Number of different words:", words.word.unique().shape[0])
grouped = words.groupby('word')
grouped.count().sort_values(by='id', ascending=False).head(10)

Number of different words: 9548


,id
word,
,28939
music,7922
country,6962
summer,6518
chill,6135
songs,6099
rock,5628
playlist,5002
good,4885


Максимальная длина слова из названия

In [21]:
max_len = 0
max_word = words_in_names[0]
for word in words_in_names:
    if len(word) > max_len:
        max_len = len(word)
        max_word = word
print ('Max len of word:', max_len, '\nword:', max_word)

Max len of word: 21 
word: classicalinstrumental


In [22]:
from nltk.corpus import stopwords

In [23]:
stop_words = stopwords.words('english') + ['music', 'song', 'songs', 'playlist', '', 'good']

In [24]:
words_in_names = np.array([word for word in words_in_names if word not in stop_words])
words = pd.DataFrame({'word': words_in_names, 'id':np.arange(words_in_names.shape[0])})
words.head()

,id,word
0,0,rap
1,1,wedding
2,2,mix
3,3,party
4,4,welp


In [25]:
print ("Number of different words:", words.word.unique().shape[0])
grouped = words.groupby('word')
grouped.count().sort_values(by='id', ascending=False).head()

Number of different words: 9431


,id
word,
country,6962
summer,6518
chill,6135
rock,5628
jams,4805


In [26]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=False)
words_stem = np.array([stemmer.stem(word) for word in words_in_names])
words = pd.DataFrame({'word': words_stem, 'id':np.arange(words_stem.shape[0])})
words.head()

,id,word
0,0,rap
1,1,wed
2,2,mix
3,3,parti
4,4,welp


In [27]:
print ("Number of different words:", words.word.unique().shape[0])
grouped = words.groupby('word')
grouped.count().sort_values(by='id', ascending=False).head()

Number of different words: 8415


,id
word,
countri,6962
summer,6518
chill,6357
rock,5676
jam,5534


In [28]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
words_lemm = np.array([wordnet_lemmatizer.lemmatize(word, pos='n') for word in words_in_names])
words = pd.DataFrame({'word': words_lemm, 'id':np.arange(words_lemm.shape[0])})
words.head()

,id,word
0,0,rap
1,1,wedding
2,2,mix
3,3,party
4,4,welp


In [29]:
print ("Number of different words:", words.word.unique().shape[0])
grouped = words.groupby('word')
grouped.count().sort_values(by='id', ascending=False).head()

Number of different words: 8932


,id
word,
country,6962
summer,6518
chill,6168
rock,5658
jam,5515


In [41]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_playlist_name_matrix(playlists):
    playlists['name'] = [str(playlists['name'].values[i]).lower() 
                         for i in range(playlists.shape[0])]
    names = playlists['name'].values
    for i in range(names.shape[0]):
        names[i] = re.compile('[^a-zA-Z0-9 ]').sub('', names[i])
    playlists['name'] = names
    words_in_names = []
    for i in range(names.shape[0]):
        words_in_names += names[i].split(' ')
    stop_words = stopwords.words('english') + ['music', 'song', 'songs', 'playlist', '', 'good']
    words_in_names = np.array([word for word in words_in_names if word not in stop_words])
    bag_of_words = CountVectorizer()
    bag_of_words.vocabulary_ = words_in_names
    print(words_in_names)
    X = bag_of_words.fit_transform(playlists.name.values)
    return X

In [42]:
playlists = pd.read_csv('{}/playlists.csv'.format(data_dir), index_col=0)
playlists.head()

,pid,name,num_followers
0,822000,rap,1
1,822001,wedding mix,1
2,822002,Party,1
3,822003,wElP,1
4,822004,Worship,1


In [43]:
X = get_playlist_name_matrix(playlists)

['rap' 'wedding' 'mix' ... 'jams' 'fall' '2014']
